In [1]:
import os, sys

def is_root(path):
    """
    Determine whether the path passed is the root directory for the file system
    """
    return not os.path.split(os.path.normpath(os.path.abspath(path)))[1]

def is_root_project_dir(path):
    """
    Determine whether the path specified is the root project directory by looking for 
    __mtk__.py in the directory. If the root file system directory is encountered, throw
    an exception
    """
    PROJECT_ROOT_FILE = '__mtk__.py'
    
    # make sure the path is in an OK format (ends with a directory separator)
    search_path = str(path)
    if (not search_path.endswith(os.sep)):
        search_path += os.sep
        
    # make sure the path isn't the root. if it is, there's a problem
    if (is_root(search_path)):
        raise Exception('Failed to locate root MTK directory (directory containint %s)' % PROJECT_ROOT_FILE)
        
    # check to see whether we can find the project root file in this path
    return os.path.isfile(search_path + PROJECT_ROOT_FILE)

def get_project_root():
    """
    Locate the project's root directory and return its location. If the root file system
    directory is encountered, throw an exception
    """
    search_path = os.getcwd() + os.sep
    count = 0
    while (not is_root_project_dir(search_path)):
        search_path = os.path.normpath(os.path.abspath(search_path + '..' + os.sep)) + os.sep
    return search_path

lib_path = get_project_root()
sys.path.insert(0, lib_path)

from lib.util import Singleton

Tests
------

In [2]:
from tests.chamber import PressureVesselTestCaseDataset
from lib.pint_ext import PintExtUnitRegistry
units = PintExtUnitRegistry()
units.default_format = '~L'

class PressureVesselTestCaseDataHtml():
    def __init__(self):
        self._dataset = PressureVesselTestCaseDataset()
    
    def _repr_html_(self):
        html = ["<table>"]
        html.append("<tr>")
        html.append("<th>Test Case</th>")
        html.append("<th>r<sub>i</sub></th>")
        html.append("<th>P<sub>c</sub></th>")
        html.append("<th>P<sub>amb</sub></th>")
        html.append("<th>&sigma;<sub>y</sub></th>")
        html.append("<th>t<sub>calc</sub></th>")
        html.append("<th>t<sub>exp</sub></th>")
        html.append("</tr>")
        for data in self._dataset:
            html.append("<tr>")
            html.append("<td>%s</td>" % data.name)
            html.append("<td>%s</td>" % (self._display_float(data.ri.to(units.millimeter), '{0:.1f}')))
            html.append("<td>%s</td>" % (self._display_float(data.p_c.to(units.kilopascal), '{0:.1f}')))
            html.append("<td>%s</td>" % (self._display_float(data.p_amb.to(units.kilopascal), '{0:.1f}')))
            html.append("<td>%s</td>" % (self._display_float(data.material_strength.to(units.megapascal), '{0:.1f}')))
            html.append("<td>%s</td>" % (self._display_float(data.t_calc.to(units.millimeter), '{0:.1f}')))
            html.append("<td>%s</td>" % (self._display_float(data.t_expected.to(units.millimeter), '{0:.1f}')))
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)
        
    def _display_float(self, x, format_string):
        return self._display_float_magnitude(x, format_string) + ' ' + self._display_units(x)

    def _display_float_magnitude(self, x, format_string):
        value = x.magnitude
        return '$' + format_string.format(float(x.magnitude)) + '$'

    def _display_units(self, x):
        pint_string_elements = '{:}'.format(x).split(' ')
        return '$' + pint_string_elements[1] + '$'
    
table = PressureVesselTestCaseDataHtml()
table

Test Case,ri,Pc,Pamb,σy,tcalc,texp
"INCONEL 718, run 63",$133.1$ $mm$,$9192.7$ $kPa$,$0.0$ $kPa$,$1075.6$ $MPa$,$1.7$ $mm$,$1.7$ $mm$
"INCONEL 718, run 57",$133.1$ $mm$,$29206.5$ $kPa$,$0.0$ $kPa$,$1075.6$ $MPa$,$5.5$ $mm$,$5.5$ $mm$
"INCONEL 718, run 55",$242.4$ $mm$,$28443.4$ $kPa$,$0.0$ $kPa$,$1075.6$ $MPa$,$9.8$ $mm$,$9.8$ $mm$
"Alloy 188, run 45",$133.1$ $mm$,$9192.7$ $kPa$,$0.0$ $kPa$,$289.6$ $MPa$,$6.5$ $mm$,$6.5$ $mm$
"Alloy 188, run 40",$242.4$ $mm$,$17812.1$ $kPa$,$0.0$ $kPa$,$289.6$ $MPa$,$23.5$ $mm$,$23.5$ $mm$
"Alloy 188, run 37",$242.4$ $mm$,$28443.4$ $kPa$,$0.0$ $kPa$,$289.6$ $MPa$,$38.8$ $mm$,$38.8$ $mm$
"Ox-Free Copper, run 18",$133.1$ $mm$,$9192.7$ $kPa$,$0.0$ $kPa$,$390.2$ $MPa$,$4.8$ $mm$,$4.8$ $mm$
"Ox-Free Copper, run 11",$196.7$ $mm$,$30003.5$ $kPa$,$0.0$ $kPa$,$319.9$ $MPa$,$29.9$ $mm$,$29.9$ $mm$
"Ox-Free Copper, run 13",$242.4$ $mm$,$17812.1$ $kPa$,$0.0$ $kPa$,$120.0$ $MPa$,$61.6$ $mm$,$61.6$ $mm$
